In [1]:
# -*- coding: utf-8 -*-
from ekonlpy.sentiment import MPCK

import pandas as pd
import multiprocessing as mp
import pickle

import os, sys
import re

In [2]:
print(sys.stdout.encoding)
print(sys.stdin.encoding)

UTF-8
cp1252


# Tidy BOK minutes text

Code Attribution: 김현진T, Fininsight

In [3]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [4]:
def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

### Korean stop words from here: 
https://www.ranks.nl/stopwords/korean

In [5]:
def getStopwords(data_path) :
    f = open(data_path, 'r',-1,'utf-8')
    stop_word = []

    while True:
        line = f.readline()
        stop_word.append(line.strip())
        if not line: break

    return stop_word 

In [6]:
data_path = './data_files/stop_wd.txt'

In [7]:
stop_word = getStopwords(data_path)
stop_word[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

In [8]:
def text2ngram(text) :
    mpck = MPCK()
    
    tokens = mpck.tokenize(text)
    tokens = [token for token in tokens if len(token.split('/')[0])>1]
    tokens = [token for token in tokens if token.split('/')[0] not in stop_word]
    
    ngrams = mpck.ngramize(tokens)

    return tokens+ngrams

## Test code: 어떤 식으로 파싱되는지 확인. 

In [9]:
source_folder="./data_files/BOK_minutes/pdf2txt/"
output_file="./data_files/BOK_minutes/pkl/"

txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
txt_files = [f for f in txt_files if f[-4:] == '.txt']
txt_files.sort()

In [10]:
df = pd.DataFrame(columns=['date', 'minutes'])
df["Economic Situation"] = ""
df["Foreign Currency"] = ""
df["Financial Markets"] = ""
df["Monetary Policy"] = ""
df["Participant Views"] = ""
df["Government View"] = ""

df["Economic Situation count"] = ""
df["Foreign Currency count"] = ""
df["Financial Markets count"] = ""
df["Monetary Policy count"] = ""
df["Participant Views count"] = ""
df["Government View count"] = ""
df['ngrams'] =""

### 한 문서의 ngram의 갯수가 매우 많다. 이를 모두 각각 하나의 row로 만들면 df의 길이가 매우 커진다. 

* 0번째 문서의 모든 ngram을 ,(comma)로 separate한 경우 (원본코드의 방법)

In [11]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for section_text in section_texts[1:3] :
            for sentence in section_text :
                ngrams.append(','.join(text2ngram(sentence)))
    
    print(ngrams)
        #print('@@@'.join(ngrams))
        #break

    df.loc[len(df)] = [txt_file.split('_-_')[0],
                       txt,
                       '@@@'.join(section_texts[0]),
                       '@@@'.join(section_texts[1]),
                       '@@@'.join(section_texts[2]),
                       '@@@'.join(section_texts[3]),
                       '@@@'.join(section_texts[4]),
                       '@@@'.join(section_texts[5]),
                       len(section_texts[0]),
                       len(section_texts[1]),
                       len(section_texts[2]),
                       len(section_texts[3]),
                       len(section_texts[4]),
                       len(section_texts[5]),
                       '@@@'.join(ngrams)]

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
['경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,하락/NNG,이유/NNG,대해/VV,위안/NNG,상시/NNG,통화/NNG,절상/NNG,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,못하/VX,외환/NNG,수급/NNG,공급우위/NNG,상태/NNG,지속/NNG,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,기업/NNG,기대/NNG,원화/NNG,절상/NNG,편향/NNG,기인/NNG,설명/NNG,엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/NNG;절상/NNG,전망/NNG;상향/NNG', '환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/NNG,움직이/VV,감안/NNG,원엔/NNG,환율/NNG,하락/NNG,일시적/VAX,현상/NNG,장기/NNG,반전/NNG,가능성/NNG,보인다는/VV,견해/NNG,밝혔으며/VV,대해/VV,견해/NNG,동의/NNG,시장/NNG,규모/NNG,일시적/VAX,외부충격/NNG,의하/VV,영향/NNG,설명/NNG,내외/NNG,금리/NNG,역전/NNG,나타나/VV,대규모/NNG,외환/NNG,유입/NNG,일어나/VV,감안/NNG,원엔/NNG,환율/NNG,추가/NNG,하락/NNG,점차/MAG,축소/NNG,생각/NNG,답변/NNG,원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일시적/VAX', '유로존/NNG,경제/NNG,성장/NNG,실적/NNG,예상/NNG,저조/NNG,가운/NNG,일각/NNG,금리/NNG,인하/NNG,대두/NNG,유로/NNG,절하/NNG,유로/NNG,달러/NNG,흐름/NNG,장기/NNG,변화/NNG,대해/VV,국제투자은행/NNG,달러/NNG,대비/NNG,유로/NNG

In [15]:
b = [3,4,5]
a = [1,2,3]
b += a
b

[3, 4, 5, 1, 2, 3]

In [47]:
df.memory_usage(index=True, deep=True)

Index                           43248
date                           362202
minutes                     230598336
Economic Situation           44804928
Foreign Currency             12055380
Financial Markets           166666980
Monetary Policy                318954
Participant Views              318954
Government View                318954
Economic Situation count       194616
Foreign Currency count         194616
Financial Markets count        194616
Monetary Policy count          172992
Participant Views count        172992
Government View count          172992
ngrams                         942580
dtype: int64

In [48]:
df[['date', 'ngrams']].memory_usage(index=True, deep=True).sum()

1348030

In [49]:
df

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngrams
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,"경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NN..."
1,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,경제/NNG@@@경제/NNG@@@비해/VV@@@특별/NNG@@@요인/NNG@@@불구...
2,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,환율/NNG@@@장기/NNG@@@각국/NNG@@@기초/NNG@@@경제/NNG@@@여...
3,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,유로존/NNG@@@경제/NNG@@@성장/NNG@@@실적/NNG@@@예상/NNG@@@...
4,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,금리/NNG@@@수준/NNG@@@유지/NNG@@@금리/NNG@@@상승/NNG@@@가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5401,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,프@@@리@@@미@@@엄@@@/@@@N@@@N@@@G
5402,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,수@@@준@@@/@@@N@@@N@@@G
5403,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,유@@@지@@@/@@@N@@@N@@@G
5404,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,,,,19,4,160,0,0,0,양@@@호@@@/@@@N@@@N@@@G


In [50]:
print(len(df.ngrams[0]))
df.ngrams[0][:300]

21924


'경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,하락/NNG,이유/NNG,대해/VV,위안/NNG,상시/NNG,통화/NNG,절상/NNG,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,못하/VX,외환/NNG,수급/NNG,공급우위/NNG,상태/NNG,지속/NNG,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,기업/NNG,기대/NNG,원화/NNG,절'

* 0번째 문서의 모든 ngram을 각각 row로 만들어 df에 담았을 경우

In [51]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for section_text in section_texts[1:3] :
            for sentence in section_text :
                ngram= text2ngram(sentence)
                for item in ngram:
                    df.loc[len(df)] = [txt_file.split('_-_')[0],
                               txt,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(item)]
        

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
Wall time: 1min 31s


In [52]:
df.memory_usage(index=True, deep=True)

Index                           64200
date                           537675
minutes                     342314400
Economic Situation           66511200
Foreign Currency             17895750
Financial Markets           247410750
Monetary Policy                473475
Participant Views              473475
Government View                473475
Economic Situation count       288900
Foreign Currency count         288900
Financial Markets count        288900
Monetary Policy count          256800
Participant Views count        256800
Government View count          256800
ngrams                        1292960
dtype: int64

In [53]:
df[['date', 'ngrams']].memory_usage(index=True, deep=True).sum()

1894835

In [9]:
def preprocessing(source_folder="./data_files/BOK_minutes/pdf2txt/", output_file="./data_files/BOK_minutes/pkl/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    txt_files = [f for f in txt_files if f[-4:] == '.txt']
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""
    df['ngrams'] =""
    

    for txt_file in txt_files :
        try :
            with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
                txt = f.read()
                sections, section_texts = preprocess_minutes(txt)

                print(txt_file)

                ngrams = []
                for section_text in section_texts[1:3] :
                    for sentence in section_text :
                        ngrams.append(','.join(text2ngram(sentence)))

                #print('@@@'.join(ngrams))
                #break

                df.loc[len(df)] = [txt_file.split('_-_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),
                                   '@@@'.join(ngrams)]
        except :
            print('Error occurred at {}'.format(txt_file))
            print('Error: ', sys.exc_info()[0])
            pass
    
    df.to_pickle(output_file)
        
    print(df.shape)
    print("---")
    print(df.head())

In [ ]:
preprocessing()

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
